In [1]:
import pandas as pd
import numpy as np

In [2]:
from Bio.Seq import Seq
from Bio import SeqIO
#from Bio.Alphabet import IUPAC
import sys
# import pyensembl
import os
#https://towardsdatascience.com/a-simple-guide-to-command-line-arguments-with-argparse-6824c30ab1c3
import warnings
warnings.filterwarnings("ignore")
from Bio.SeqUtils import seq3

In [5]:
# SK: Dictionary of protein names to sequences
proteins = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_translations.fa", "fasta"):
    name = record.id.split("|")[1].split(".")[0]
    proteins[name] = str(record.seq)
    

# SK: Dictionary of protein names to CDS dna transcript

dna_transcripts = {}
for record in SeqIO.parse("../raw_files/gencode.v36.pc_transcripts.fa", "fasta"):
	name = record.id.split("|")[0].split(".")[0]
	record_c = record.id.split("|")
	for i in record_c:
		if "CDS" in i:
			coords = i.replace("CDS:","")
	start = int(coords.split("-")[0])
	end = int(coords.split("-")[1])
	dna_seq = str(record.seq)[start-1:end]
	dna_transcripts[name] = dna_seq
    
uniprotID_ENST_mapping = pd.read_csv("../../data/SFARI_TFs_with_ENST_corrected.csv")
uniprotID_ENST_mapping = uniprotID_ENST_mapping[["uniprotID", "ENST"]]
uniprotID_ENST_mapping["ENST"] = uniprotID_ENST_mapping["ENST"].str.split(".").str[0]
uniprotID_ENST_mapping_dict= dict(zip(uniprotID_ENST_mapping["uniprotID"],uniprotID_ENST_mapping["ENST"]))
uniprotID_ENST_mapping_dict['O60479'] = 'ENST00000434704'

In [6]:
# uniprotID_ENST_mapping_dict['P11308'] = 'ENST00000288319'

In [7]:
dna_transcripts

{'ENST00000641515': 'ATGAAGAAGGTAACTGCAGAGGCTATTTCCTGGAATGAATCAACGAGTGAAACGAATAACTCTATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAACTCCAGACCTTCCTATTTATGTTGTTTTTTGTATTCTATGGAGGAATCGTGTTTGGAAACCTTCTTATTGTCATAACAGTGGTATCTGACTCCCACCTTCACTCTCCCATGTACTTCCTGCTAGCCAACCTCTCACTCATTGATCTGTCTCTGTCTTCAGTCACAGCCCCCAAGATGATTACTGACTTTTTCAGCCAGCGCAAAGTCATCTCTTTCAAGGGCTGCCTTGTTCAGATATTTCTCCTTCACTTCTTTGGTGGGAGTGAGATGGTGATCCTCATAGCCATGGGCTTTGACAGATATATAGCAATATGCAAGCCCCTACACTACACTACAATTATGTGTGGCAACGCATGTGTCGGCATTATGGCTGTCACATGGGGAATTGGCTTTCTCCATTCGGTGAGCCAGTTGGCGTTTGCCGTGCACTTACTCTTCTGTGGTCCCAATGAGGTCGATAGTTTTTATTGTGACCTTCCTAGGGTAATCAAACTTGCCTGTACAGATACCTACAGGCTAGATATTATGGTCATTGCTAACAGTGGTGTGCTCACTGTGTGTTCTTTTGTTCTTCTAATCATCTCATACACTATCATCCTAATGACCATCCAGCATCGCCCTTTAGATAAGTCGTCCAAAGCTCTGTCCACTTTGACTGCTCACATTACAGTAGTTCTTTTGTTCTTTGGACCATGTGTCTTTATTTATGCCTGGCCATTCCCCATCAAGTCATTAGATAAATTCCTTGCTGTATTTTATTCTGTGATCACCCCTCTCTTGAACCCAATTATATACACACTGAGGAACAAAGACATGAAGACGGCAATAAGACAGCTGAGAAAATGGGATGCACATTCTAGTGTAAAGTTTT

# Plan:
1. Use cds coordinates, create df with genomic coordinates as one column and corresponding nt as second column
2. Use position in variant file to resplace the value in the nt column
3. Translate the nt column

In [12]:
directory = "../outputs/mutations/domains_bed_format/"
files = os.listdir(directory)

In [14]:
def save_variant_fasta(uniprotID, save = True):
    if uniprotID in uniprotID_ENST_mapping_dict.keys():
        # finding ENST corresponding to uniprotID, reading in missense SNP variants
        ENST = uniprotID_ENST_mapping_dict[uniprotID]
        variants = pd.read_csv("../outputs/mutations/domains_expanded_iWES_v2_variants_snv_classified/" + ENST + ".bed", sep = "\t", header = None)
        # display(variants)
        no_syn_variants = variants[variants[22] == "No-Syn"]
        no_syn_variants = no_syn_variants[no_syn_variants[3] == "AD"]
        # no_syn_variants = variants[variants[3] == "AD"] SK: 3/14- do I need this
        
        # Finding strand
        strand = no_syn_variants[13].iloc[1]

        # Finding wt nucleotide sequence of transcript, and corresponding AA sequence
        wt_nt_seq = dna_transcripts[ENST]
        wt_AA_seq = str(Seq(wt_nt_seq).translate())
        display(wt_AA_seq)

        if strand == "-":
            wt_nt_seq = str(Seq(wt_nt_seq).complement())

        nt_df = pd.DataFrame({"nt" : [*wt_nt_seq]})
        nt_df["pred_prot_pos"] = [i for i in range(1, int((len(nt_df)) / 3 + 1)) for _ in range(3)]
    
        cds_bed = pd.read_csv("../outputs/mutations/cds_bed_format/" + ENST, sep = "\t", header = None)
        cds_bed = cds_bed[[1, 2]]
        cds_bed[1] += 1

        if strand == "-":
            cds_bed = cds_bed.sort_values(by = 1, ascending = False)
        else:
            cds_bed = cds_bed.sort_values(by = 1, ascending = True)

        range_col = []
        for start, end in zip(cds_bed[1], cds_bed[2]):
            if strand == "-":
                range_col += list(range(end, start - 1, -1))
            else:
                range_col += list(range(start, end + 1))

        nt_df["gen_pos"] = range_col
        nt_df = nt_df.set_index("gen_pos")


        # Checking 
        for i in no_syn_variants.index:
            pos = no_syn_variants[2].loc[i]
            sfari_nt = no_syn_variants[no_syn_variants[2] == pos][17].loc[i]
            nt_df_nt = nt_df.loc[pos].iloc[0]
            if sfari_nt != nt_df_nt:
                print("mismatch!")

        # display(no_syn_variants)
        no_syn_variants["name"] = "g." + no_syn_variants[0].astype(str) + "."+ no_syn_variants[2].astype(str) + no_syn_variants[17] + ">" + no_syn_variants[18]
        
        names = []
        TF_seqs = []
        prot_positions = []


        for i in no_syn_variants.index:
            var_pos = no_syn_variants[2].loc[i]
            wt_nt = no_syn_variants[17].loc[i]
            var_nt = no_syn_variants[18].loc[i]
            g_name = no_syn_variants["name"].loc[i]

            nt_df_var_copy = nt_df.copy(deep = True)

            if nt_df_var_copy.at[var_pos, "nt"] == wt_nt:
                nt_df_var_copy.at[var_pos, "nt"] = var_nt
            else:
                print(uniprotID + " mismatch!")

            new_nt_seq = "".join(nt_df_var_copy["nt"])
            if strand == "-":
                new_nt_seq = Seq(new_nt_seq).complement()
            new_AA_seq = str(Seq(new_nt_seq).translate())

            # prot_pos = 0
            for i in range(len(new_AA_seq)):
                if new_AA_seq[i] != wt_AA_seq[i]:
                    prot_change_descrip = "p." + seq3(wt_AA_seq[i]) + str(i + 1)+ seq3(new_AA_seq[i])
                    prot_positions.append(i + 1)
                    names.append(g_name + "(" + prot_change_descrip + ")")
                    TF_seqs.append(new_AA_seq)

                # prot_pos = i + 1    

                
            
        variant_TF_seqs_df = pd.DataFrame({"name": names,
                     "TF_seq": TF_seqs, 
                    "prot_pos":prot_positions})

        def return_AD_seqs(AD_start, AD_end):
            print(AD_start)
            print(AD_end)
            #display(variant_TF_seqs_df)
            within_AD = variant_TF_seqs_df[(AD_start <= variant_TF_seqs_df["prot_pos"]) & (variant_TF_seqs_df["prot_pos"] <= AD_end)]
            #display(within_AD)
            within_AD["AD_seq"] = within_AD["TF_seq"].str[AD_start - 1: AD_end]
            #display(within_AD)
            return within_AD

        # return_AD_seqs(340, 477)

        known_ADs = pd.read_csv("../../output/known_ADs_considering_isoforms_and_canonical.csv")
        AD_rows = known_ADs[known_ADs["uniprotID"] == uniprotID]

        gene = AD_rows["Gene"].iloc[0].replace(" ", "").replace("/", "-")

        def save_fasta(start, end):
            df = return_AD_seqs(start, end)
            df = df.reset_index()
            display(df)
            ofile = open("../outputs/AD_variant_fasta/" + gene + "_" + uniprotID + "_" + "AD_" + str(start) + "-" + str(end), "w")
            for i in df.index:
                ofile.write(">" + df["name"].loc[i] + "\n" + df["AD_seq"].loc[i] + "\n")
            ofile.close()
            
    
        # display(AD_rows)

        if save:
            for start, end in zip(AD_rows["Start"], AD_rows["End"]):
                save_fasta(start, end)
        
        return variant_TF_seqs_df


In [18]:
for uniprotID in list(files):
    save_variant_fasta(uniprotID)

'MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEKRMSKDEERAVKDELLGEKPEIKQKWASRLLAKLRKDIRPEFREDFVLTITGKKPPCCVLSNPDQKGKIRRIDCLRQADKVWRLDLVMVILFKGIPLESTDGERLYKSPQCSNPGLCVQPHHIGVTIKELDLYLAYFVHTPESGQSDSSNQQGDADIKPLPNGHLSFQDCFVTSGVWNVTELVRVSQTPVATASGPNFSLADLESPSYYNINQVTLGRRSITSPPSTSTTKRPKSIDDSEMESPVDDVFYPGTGRSPAAGSSQSSGWPNDVDAGPASLKKSGKLDFCSALSSQGSSPRMAFTHHPLPVLAGVRPGSPRATASALHFPSTSIIQQSSPYFTHPTIRYHHHHGQDSLKEFVQFVCSDGSGQATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDGAALTPPSPSFATTGASSANRFVSIGPRDGNFLNIPQQSQSWFL*'

414
502


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.19.13081853C>G(p.Gln418Glu),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,418,QATGEPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDG...
1,1,g.19.13088001G>A(p.Gly423Ser),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,423,QATGQPNGSSQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDG...
2,2,g.19.13088017C>T(p.Pro428Leu),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,428,QATGQPNGSGQGKVLGSFLLPPPPPVARPVPLPMPDSKSTSTAPDG...
3,3,g.19.13088038C>T(p.Pro435Leu),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,435,QATGQPNGSGQGKVPGSFLLPLPPPVARPVPLPMPDSKSTSTAPDG...
4,4,g.19.13088049G>A(p.Val439Met),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,439,QATGQPNGSGQGKVPGSFLLPPPPPMARPVPLPMPDSKSTSTAPDG...
5,5,g.19.13088058C>G(p.Pro442Ala),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,442,QATGQPNGSGQGKVPGSFLLPPPPPVARAVPLPMPDSKSTSTAPDG...
6,6,g.19.13088092C>T(p.Thr453Ile),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,453,QATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSISTAPDG...
7,7,g.19.13088097A>G(p.Thr455Ala),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,455,QATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSAAPDG...
8,8,g.19.13088104C>T(p.Pro457Leu),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,457,QATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTALDG...
9,9,g.19.13088118T>A(p.Leu462Met),MYSPYCLTQDEFHPFIEALLPHVRAFSYTWFNLQARKRKYFKKHEK...,462,QATGQPNGSGQGKVPGSFLLPPPPPVARPVPLPMPDSKSTSTAPDG...


'MAQRYDELPHYGGMDGVGVPASMYGDPHAPRPIPPVHHLNHGPPLHATQHYGAHAPHPNVMPASMGSAVNDALKRDKDAIYGHPLFPLLALVFEKCELATCTPREPGVAGGDVCSSDSFNEDIAVFAKQVRAEKPLFSSNPELDNLMIQAIQVLRFHLLELEKVHELCDNFCHRYISCLKGKMPIDLVIDERDGSSKSDHEELSGSSTNLADHNPSSWRDHDDATSTHSAGTPGPSSGGHASQSGDNSSEQGDGLDNSVASPGTGDDDDPDKDKKRQKKRGIFPKVATNIMRAWLFQHLTHPYPSEEQKKQLAQDTGLTILQVNNWFINARRRIVQPMIDQSNRAGFLLDPSVSQGAAYSPEGQPMGSFVLDGQQHMGIRPAGLQSMPGDYVSQGGPMGMSMAQPSYTPPQMTPHPTQLRHGPPMHSYLPSHPHHPAMMMHGGPPTHPGMTMSAQSPTMLNSVDPNVGGQVMDIHAQ*'

340
477


...

13
214


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.8.22691018G>C(p.Pro207Ala),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHY...,207,MSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
1,1,g.8.22691019A>C(p.Ile206Met),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHY...,206,MSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
2,2,g.8.22691021T>A(p.Ile206Phe),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHY...,206,MSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
3,3,g.8.22691023G>A(p.Ser205Phe),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHY...,205,MSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
4,4,g.8.22691024A>C(p.Ser205Ala),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHY...,205,MSSLLNQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
...,...,...,...,...,...
56,56,g.8.22692854T>C(p.Ser31Gly),MTGKLAEKLPVTMSSLLNQLPDNLYPEEIPGALNLFSGSSDSVVHY...,31,MSSLLNQLPDNLYPEEIPGALNLFSGSSDSVVHYNQMATENVMDIG...
57,57,g.8.22692866C>T(p.Glu27Lys),MTGKLAEKLPVTMSSLLNQLPDNLYPKEIPSALNLFSGSSDSVVHY...,27,MSSLLNQLPDNLYPKEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
58,58,g.8.22692869G>A(p.Pro26Ser),MTGKLAEKLPVTMSSLLNQLPDNLYSEEIPSALNLFSGSSDSVVHY...,26,MSSLLNQLPDNLYSEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...
59,59,g.8.22692891G>C(p.Asn18Lys),MTGKLAEKLPVTMSSLLKQLPDNLYPEEIPSALNLFSGSSDSVVHY...,18,MSSLLKQLPDNLYPEEIPSALNLFSGSSDSVVHYNQMATENVMDIG...


'MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKYARHRSVYGCPLAKKRKTQDKQPQEPAPKRKPFAVKADSSSVDECDDSDGTEDMDEKEEDEGEEYSEDNDEPGDEDEEDEEGDREEEEEIEEEDEDDDEDGEDVEDEEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNYDELVAKSLLNLGKIAEDAAYRARTESEMNSNTSNSLEDDSDKNENLGRKSELSLDLDSDVVRETVDSLKLLAQGHGVVLSENMNDRNYADSMSQQDSRNMNYVMLGKPMNNGLMEKMVEESDEEVCLSSLECLRNQCFDLARKLSETNPQERNPQQNMNIRQHVRPEEDFPGRTPDRNYSDMLNLMRLEEQLSPRSRVFASCAKEDGCHERDDDTTSVNSDRSEEVFDMTKGNLTLLEKAIALETERAKAMREKMAMEAGRRDNMRSYEDQSPRQLPGEDRKPKSSDSHVKKPYYGKDPSRTEKKESKCPTPGCDGTGHVTGLYPHHRSLSGCPHKDRVPPEILAMHESVLKCPTPGCTGRGHVNSNRNSHRSLSGCPIAAAEKLAKAQEKHQSCDVSKSSQASDRVLRPMCFVKQLEIPQYGYRNNVPTTTPRSNLAKELEKYSKTSFEYNSYDNHTYGKRAIAPKVQTRDISPKGYDDAKRYCKDPSPSSSSTSSYAPSSSSNLSCGGGSSASSTCSKSSFDYTHDMEAAHMAATAILNLSTRCREMPQNLSTKPQDLCATRNPDMEVDENGTLDLSMNKQRPRDSCCPILTPLEPMSPQQQAVMNNRCFQLGEGDCWDLPVDYTKMKPRRIDEDESKDITPEDLDPFQEALEERRYPGEVTIPSPKPKYPQCKESKKDLITLSGCPLADKSIRSMLATSSQELKCPTPGCDGSGHITGNYASHRSLSGCPRAKKSGIRIAQSKEDKEDQEPIRCPVPGCDGQGHITGKYASHRSASGCPLAAKRQKDGYLNGSQFSWKS

152
425


,index,name,TF_seq,prot_pos,AD_seq
0,0,g.2.1922496C>T(p.Asp425Asn),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,425,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
1,1,g.2.1922502C>A(p.Val423Leu),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,423,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
2,2,g.2.1922548C>A(p.Glu407Asp),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,407,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
3,3,g.2.1922558C>A(p.Gly404Val),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,404,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
4,4,g.2.1922558C>G(p.Gly404Ala),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,404,EEEEEEEEEEEEEEEENEDHQMNCHNTRIMQDTEKDDNNNDEYDNY...
...,...,...,...,...,...
64,64,g.2.1923226C>T(p.Met181Ile),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,181,EEEEEEEEEEEEEEEENEDHQMNCHNTRIIQDTEKDDNNNDEYDNY...
65,65,g.2.1923227A>C(p.Met181Arg),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,181,EEEEEEEEEEEEEEEENEDHQMNCHNTRIRQDTEKDDNNNDEYDNY...
66,66,g.2.1923240T>C(p.Asn177Asp),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,177,EEEEEEEEEEEEEEEENEDHQMNCHDTRIMQDTEKDDNNNDEYDNY...
67,67,g.2.1923242T>C(p.His176Arg),MEVDTEEKRHRTRSKGVRVPVEPAIQELFSCPTPGCDGSGHVSGKY...,176,EEEEEEEEEEEEEEEENEDHQMNCRNTRIMQDTEKDDNNNDEYDNY...
